<a href="https://colab.research.google.com/github/Ernawaty2024/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [3]:
# Check the variance of each numerical column
for column in application_df.select_dtypes(include=['number']).columns:
    print(f"{column}: {application_df[column].var()}")

STATUS: 0.0001457598435995139
ASK_AMT: 7591715741693088.0
IS_SUCCESSFUL: 0.2489570971171259


In [4]:
# Determine the number of unique values in each column.
print(application_df.nunique().to_string(header=False))


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2


### Preprocessing 'ASK_AMT' column to bin it


In [5]:
# Find minimun value of ASK_AMT
min_ask_amt = application_df['ASK_AMT'].min()
print(min_ask_amt)

5000


In [6]:
# Find maximum value of ASK_AMT
max_ask_amt = application_df['ASK_AMT'].max()
print(max_ask_amt)

8597806340


In [7]:
# Define bin_edges
bin_edges = [min_ask_amt,5000,15000,50000, 100000,500000,1000000,5000000, max_ask_amt]

# Bin the 'ASK_AMT' column
application_df['ASK_AMT_BIN'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, include_lowest=True, duplicates='drop')

# Display value counts for the binned column
print(application_df['ASK_AMT_BIN'].value_counts().to_string(header=False))

(4999.999, 15000.0]          26334
(100000.0, 500000.0]          2304
(15000.0, 50000.0]            2011
(50000.0, 100000.0]           1423
(1000000.0, 5000000.0]         940
(500000.0, 1000000.0]          650
(5000000.0, 8597806340.0]      637


In [8]:
application_df = application_df.drop(columns = ['ASK_AMT'])

In [9]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'IS_SUCCESSFUL', 'ASK_AMT_BIN'],
      dtype='object')

In [10]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts.to_string(header=False))

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1


In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_counts[application_counts <500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_value_counts.to_string(header=False))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_to_replace = classification_value_counts[classification_value_counts > 1]
classification_to_replace

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
# Include the 'ASK_AMT_BIN' column in the get_dummies call
categorical_dummies = pd.get_dummies(application_df,
                                      columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT_BIN',
                                               'AFFILIATION', 'USE_CASE', 'ORGANIZATION',
                                               'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], drop_first=True)


In [16]:
# Split our preprocessed data into our features and target arrays
X = categorical_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = categorical_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, random_state = 42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the custom threshold callback
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs["accuracy"]  # or "val_accuracy" for validation accuracy
        if accuracy >= self.threshold:
            self.model.stop_training = True

# Ensure the directory for saving model weights exists
checkpoint_dir = "checkpoints/"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [19]:
# Define the checkpoint callback to save weights every 5 epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "weights_epoch_{epoch:02d}.weights.h5"),  # Changed the filename
    save_weights_only=True,
    verbose=1,
    save_best_only=True,  # Save only the best model
    save_freq=5  # Save every 5 epochs based on batches
)


In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\Ernie\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,821 (22.74 KB)

 Trainable params: 5,821 (22.74 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [22]:
# Define the custom callback
my_callback = MyThresholdCallback(threshold=0.75)

# Train the model with the custom callback
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[my_callback])



Epoch 1/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6886 - loss: 0.6033 - val_accuracy: 0.7198 - val_loss: 0.5711
Epoch 2/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7284 - loss: 0.5549 - val_accuracy: 0.7265 - val_loss: 0.5656
Epoch 3/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7293 - loss: 0.5546 - val_accuracy: 0.7257 - val_loss: 0.5633
Epoch 4/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7294 - loss: 0.5510 - val_accuracy: 0.7245 - val_loss: 0.5621
Epoch 5/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7291 - loss: 0.5500 - val_accuracy: 0.7254 - val_loss: 0.5636
Epoch 6/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7361 - loss: 0.5465 - val_accuracy: 0.7251 - val_loss: 0.5616
Epoch 7/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7371 - loss: 0.5403 - val_accuracy: 0.7254 - val_loss: 0.5610
Epoch 8/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7342 - loss: 0.5467 - val_accu

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - 1ms/step - accuracy: 0.7157 - loss: 0.5746
Loss: 0.5745808482170105, Accuracy: 0.7157434225082397


# Compile, Train and Optimize the Model for first optimization
- add another hidden layer



In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid',))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units = 10, activation = 'tanh'))

# Output layer
nn1.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 80)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,111 (23.87 KB)

 Trainable params: 6,111 (23.87 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Compile the model
nn1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [26]:
# Train the model with the EarlyStopping callback
model_train = nn1.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks = [my_callback])

Epoch 1/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6975 - loss: 0.5953 - val_accuracy: 0.7236 - val_loss: 0.5737
Epoch 2/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7304 - loss: 0.5514 - val_accuracy: 0.7242 - val_loss: 0.5659
Epoch 3/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7276 - loss: 0.5557 - val_accuracy: 0.7233 - val_loss: 0.5677
Epoch 4/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7325 - loss: 0.5486 - val_accuracy: 0.7210 - val_loss: 0.5646
Epoch 5/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7304 - loss: 0.5499 - val_accuracy: 0.7265 - val_loss: 0.5623
Epoch 6/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7320 - loss: 0.5504 - val_accuracy: 0.7230 - val_loss: 0.5617
Epoch 7/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7351 - loss: 0.5456 - val_accuracy: 0.7239 - val_loss: 0.5638
Epoch 8/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7354 - loss: 0.5430 - val_accu

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - 1ms/step - accuracy: 0.7157 - loss: 0.5837
Loss: 0.5837205052375793, Accuracy: 0.7157434225082397


## Second Optimization
- Increase epoch from 100 to 200


In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid'))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn2.summary()

c:\Users\Ernie\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 80)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,821 (22.74 KB)

 Trainable params: 5,821 (22.74 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Compile the model
nn2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [30]:
# Train the model
model_train = nn2.fit(X_train_scaled, y_train, epochs=200, validation_data=(X_test_scaled, y_test), callbacks = [my_callback])

Epoch 1/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6864 - loss: 0.6008 - val_accuracy: 0.7230 - val_loss: 0.5686
Epoch 2/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7279 - loss: 0.5576 - val_accuracy: 0.7233 - val_loss: 0.5671
Epoch 3/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7285 - loss: 0.5534 - val_accuracy: 0.7192 - val_loss: 0.5687
Epoch 4/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7369 - loss: 0.5443 - val_accuracy: 0.7195 - val_loss: 0.5653
Epoch 5/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7341 - loss: 0.5447 - val_accuracy: 0.7248 - val_loss: 0.5619
Epoch 6/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7329 - loss: 0.5487 - val_accuracy: 0.7213 - val_loss: 0.5633
Epoch 7/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7336 - loss: 0.5467 - val_accuracy: 0.7216 - val_loss: 0.5623
Epoch 8/200
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7337 - loss: 0.5444 - val_accu

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - 1ms/step - accuracy: 0.7102 - loss: 0.5885
Loss: 0.5885274410247803, Accuracy: 0.7102040648460388


# Third Optimization
- using hyperparameter Tuning


In [33]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [35]:
!pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ------------ --------------------------- 41.0/129.1 kB ? eta -:--:--
   -------------------------------------- - 122.9/129.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 129.1/129.1 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import keras_tuner as kt

# Define the EarlyStopping callback and assing it to stop_early
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.001,
    verbose=0,
    mode='max',
    baseline=0.75,
    restore_best_weights=True)

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=200,
    factor = 3,
    directory="tune_dir",
    project_name="tune_project")

In [37]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test), callbacks = [my_callback])

Trial 142 Complete [00h 00m 37s]
val_accuracy: 0.7206997275352478

Best val_accuracy So Far: 0.7285714149475098
Total elapsed time: 00h 38m 21s

Search: Running Trial #143

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
65                |23                |first_units
4                 |2                 |num_layers
19                |27                |units_0
41                |75                |units_1
3                 |15                |units_2
47                |61                |units_3
69                |55                |units_4
17                |87                |units_5
67                |3                 |tuner/epochs
23                |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
3                 |0                 |tuner/round
0135              |None              |tuner/trial_id

Epoch 24/67
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7322 - loss: 0.563

KeyboardInterrupt: 

In [38]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 23,
 'num_layers': 2,
 'units_0': 27,
 'units_1': 75,
 'units_2': 15,
 'units_3': 61,
 'units_4': 55,
 'units_5': 87,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}